In [5]:
import pandas as pd

df = pd.read_csv('/home/xrh1/experiments/huggan/results_cyclegan_s2r/results/notebooks/huggan_repo/community-events/huggan/pytorch/cyclegan/experiments.csv', index_col=0)

In [9]:
df.columns

Index(['Unnamed: 0', 'batch_size', 'beta1', 'beta2', 'channels',
       'checkpoint_interval', 'dataset_name', 'decay_epoch', 'epoch',
       'image_size', 'lambda_cyc', 'lambda_id', 'lr', 'mixed_precision',
       'model_name', 'n_residual_blocks', 'num_epochs', 'num_workers',
       'organization_name', 'push_to_hub', 'pytorch_dump_folder_path',
       'sample_interval', 'wandb', 'output_dir'],
      dtype='object')

In [11]:
df['decay_epoch']

0     10.0
1     10.0
2     20.0
3     20.0
4     20.0
5     50.0
6     50.0
7     50.0
8     80.0
9     80.0
10    80.0
Name: decay_epoch, dtype: float64

In [14]:
df[(df['lr'] == 0.0002) & (df['decay_epoch']==80.) ]

,Unnamed: 0,batch_size,beta1,beta2,channels,checkpoint_interval,dataset_name,decay_epoch,epoch,image_size,...,model_name,n_residual_blocks,num_epochs,num_workers,organization_name,push_to_hub,pytorch_dump_folder_path,sample_interval,wandb,output_dir
8,NaN,96.0,0.5,0.999,3.0,10.0,Chris1/sim2real_gta5_to_cityscapes,80.0,0.0,64.0,...,test_cyclegan3,9.0,100.0,8.0,Chris1,True,torch_dump,50.0,True,batch_size__96--beta1__0.5--beta2__0.999--chan...


In [15]:
import argparse
import os
import numpy as np
import itertools
from pathlib import Path
import datetime
import time
import sys

from PIL import Image

from torchvision.transforms import Compose, Resize, ToTensor, Normalize, RandomCrop, RandomHorizontalFlip
from torchvision.utils import save_image, make_grid

from torch.utils.data import DataLoader

from modeling_cyclegan import GeneratorResNet, Discriminator

from utils import ReplayBuffer, LambdaLR

from datasets import load_dataset

from accelerate import Accelerator

import torch.nn as nn
import torch

In [18]:
G_AB

GeneratorResNet(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace=True)
    (10): ResidualBlock(
      (block): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
        (2): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (3): ReLU(inplace=True)
        (4): ReflectionPad2d((1, 1, 1, 1))
        

In [17]:

from collections import OrderedDict
import os
import torch
from modeling_cyclegan import GeneratorResNet, Discriminator
state_dict = torch.load(

    '/home/xrh1/Downloads/G_BA_195.pth'
    , map_location='cpu')

cleaned_state_dict = OrderedDict()
for k,v in state_dict.items():
    cleaned_state_dict[k.replace('module.','')] = v


channels = 3
image_size = 256
input_shape = (channels, image_size, image_size)
n_residual_blocks = 9
G_AB = GeneratorResNet(input_shape, n_residual_blocks)
#G_BA = GeneratorResNet(input_shape, args.n_residual_blocks)

G_AB.load_state_dict(cleaned_state_dict)

#dooggies__2__nftrex

d2 = 'dooggies'
d1 = 'nftrex'
model_name = f'{d1}__2__{d2}'
G_AB.push_to_hub(
repo_path_or_name=model_name,
    organization='Chris1')

Cloning https://huggingface.co/Chris1/nftrex__2__dooggies into local empty directory.


Upload file pytorch_model.bin:   0%|          | 32.0k/43.4M [00:00<?, ?B/s]

To https://huggingface.co/Chris1/nftrex__2__dooggies
   e6c80ee..d2a16f8  main -> main



'https://huggingface.co/Chris1/nftrex__2__dooggies/commit/d2a16f86bee8c632042f7eca86df18ad234e5ab5'

In [32]:
from collections import OrderedDict
import os
import torch
from modeling_cyclegan import GeneratorResNet, Discriminator
state_dict = torch.load(
'/home/xrh1/Downloads/s2r_256_results%20(1)/notebooks/community-events/huggan/pytorch/cyclegan/experiments_cyclegan/0/saved_models/Chris1/sim2real_gta5_to_cityscapes/G_AB_100.pth'
    #    '/home/xrh1/experiments/huggan/huggan_repo/community-events/huggan/pytorch/cyclegan/experiments/experiments_cyclegan_punk_to_apes_HQ/0/saved_models/Chris1/NFT_cryptopunks_to_bored_apes_HQ/G_AB_100.pth'
    , map_location='cpu')

cleaned_state_dict = OrderedDict()
for k,v in state_dict.items():
    cleaned_state_dict[k.replace('module.','')] = v


channels = 3
image_size = 512#256
input_shape = (channels, image_size, image_size)
n_residual_blocks = 9
G_AB = GeneratorResNet(input_shape, n_residual_blocks)
#G_BA = GeneratorResNet(input_shape, args.n_residual_blocks)

G_AB.load_state_dict(cleaned_state_dict)
G_AB.push_to_hub(
repo_path_or_name='sim2real',
    organization='Chris1')


FileNotFoundError: [Errno 2] No such file or directory: '/home/xrh1/Downloads/s2r_256_results%20(1)/notebooks/community-events/huggan/pytorch/cyclegan/experiments_cyclegan/0/saved_models/Chris1/sim2real_gta5_to_cityscapes/G_AB_110.pth'

In [ ]:
from collections import OrderedDict
import os
import torch
from modeling_cyclegan import GeneratorResNet, Discriminator
state_dict = torch.load(
    '/home/xrh1/experiments/huggan/huggan_repo/community-events/huggan/pytorch/cyclegan/experiments/experiments_cyclegan_punk_to_apes_HQ/0/saved_models/Chris1/NFT_cryptopunks_to_bored_apes_HQ/G_AB_100.pth'
    , map_location='cpu')

cleaned_state_dict = OrderedDict()
for k,v in state_dict.items():
    cleaned_state_dict[k.replace('module.','')] = v


channels = 3
image_size = 256
input_shape = (channels, image_size, image_size)
n_residual_blocks = 9
G_AB = GeneratorResNet(input_shape, n_residual_blocks)
#G_BA = GeneratorResNet(input_shape, args.n_residual_blocks)

G_AB.load_state_dict(cleaned_state_dict)
G_AB.push_to_hub(
repo_path_or_name='CycleGAN_s2r',
    organization='Chris1')


In [24]:
from collections import OrderedDict
import os
import torch
from modeling_cyclegan import GeneratorResNet, Discriminator
state_dict = torch.load(
    '/home/xrh1/experiments/huggan/huggan_repo/community-events/huggan/pytorch/cyclegan/experiments/experiments_cyclegan_punk_to_apes_HQ/0/saved_models/Chris1/NFT_cryptopunks_to_bored_apes_HQ/G_AB_80.pth'
    , map_location='cpu')

cleaned_state_dict = OrderedDict()
for k,v in state_dict.items():
    cleaned_state_dict[k.replace('module.','')] = v


channels = 3
image_size = 256
input_shape = (channels, image_size, image_size)
n_residual_blocks = 9
G_AB = GeneratorResNet(input_shape, n_residual_blocks)
#G_BA = GeneratorResNet(input_shape, args.n_residual_blocks)

G_AB.load_state_dict(cleaned_state_dict)
G_AB.push_to_hub(
repo_path_or_name='punk2ape_epoch80',
    organization='Chris1')

Cloning https://huggingface.co/Chris1/punk2ape_epoch80 into local empty directory.


Upload file pytorch_model.bin:   0%|          | 32.0k/43.4M [00:00<?, ?B/s]

To https://huggingface.co/Chris1/punk2ape_epoch80
   5b81052..6ca4d94  main -> main



'https://huggingface.co/Chris1/punk2ape_epoch80/commit/6ca4d94c2a41d30e107dcd6423c02232993b0ee7'

In [19]:
from torchvision.transforms import Compose, Resize, ToTensor, Normalize, RandomCrop, RandomHorizontalFlip
import torch
from PIL import Image

transform = Compose(
    [

       ToTensor(),
       Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
  ]
)

input = (torch.randn((256,256,3))*255).clamp(0,255).byte()
input = Image.fromarray(input.numpy())
transform(input)

tensor([[[-0.7020,  0.3804, -1.0000,  ...,  1.0000,  0.7020, -0.7098],
         [-1.0000, -0.9059, -1.0000,  ..., -1.0000,  1.0000, -1.0000],
         [-0.0510, -1.0000,  1.0000,  ..., -1.0000, -0.8353, -0.7098],
         ...,
         [-1.0000, -1.0000,  0.0275,  ...,  0.1137,  0.0745, -1.0000],
         [-1.0000, -0.2549, -1.0000,  ..., -1.0000,  0.9216, -1.0000],
         [ 1.0000,  0.9216, -1.0000,  ..., -1.0000,  1.0000, -1.0000]],

        [[ 1.0000, -1.0000,  1.0000,  ...,  0.0980,  1.0000, -1.0000],
         [-0.0196,  0.6784,  1.0000,  ...,  1.0000, -1.0000, -0.1529],
         [-0.8980, -1.0000, -0.0353,  ...,  0.8275, -0.5451,  0.3412],
         ...,
         [-1.0000,  1.0000,  1.0000,  ..., -0.0667,  1.0000, -0.6314],
         [-0.2392,  1.0000, -0.2235,  ...,  1.0000, -1.0000,  0.1137],
         [-1.0000, -0.0824, -0.7569,  ..., -1.0000, -1.0000, -1.0000]],

        [[-1.0000, -0.6392, -0.7569,  ..., -1.0000,  1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000,  ..., -1

In [28]:
input = torch.randn((4,256,256,3))

for item in input:
    print(item.shape)

torch.Size([256, 256, 3])
torch.Size([256, 256, 3])
torch.Size([256, 256, 3])
torch.Size([256, 256, 3])


In [22]:
import glob
from PIL import Image
import os
for sample in glob.glob('/home/xrh1/experiments/huggan/spaces/samples/*.png'):
    img = Image.open(sample)
    
    
    filename = os.path.basename(sample)
    dirname =  os.path.dirname(os.path.dirname(sample))
    
    half = 0.5
    out = img.resize( [int(half * s) for s in img.size] )
    
    outdir = os.path.join(dirname, 'resized_samples')
    os.makedirs(outdir, exist_ok=True)
    out.save( os.path.join(outdir,filename) )